In [1]:
def test():
    print(1)

test()

1


In [2]:
import os, json
base_path = '../clef2024-checkthat-lab/task5'
base_data_path = os.path.join(base_path, 'data')

os.listdir(base_data_path)

['.gitkeep',
 'Arabic_dev.json',
 'Arabic_train.json',
 'dev_qrels.txt',
 'English_dev.json',
 'English_train.json']

In [3]:
train_jsons = []

with open(os.path.join(base_data_path, 'English_train.json'), encoding='utf8') as file:
    for line in file:
        train_jsons += [json.loads(line)]

len(train_jsons)        

96

In [14]:
dev_jsons = []

with open(os.path.join(base_data_path, 'English_dev.json'), encoding='utf8') as file:
    for line in file:
        dev_jsons += [json.loads(line)]

len(dev_jsons)      

32

In [15]:
# display(dev_jsons)

In [16]:
with open('temp-data/eng-train.jsonl', mode='w') as file:
    file.write('')

with open('temp-data/eng-train.jsonl', mode='a', encoding='utf8') as file:
    for tweet in train_jsons[0]['timeline']:
        o = {'id': tweet[1], 'contents': tweet[2]}
        file.write(json.dumps(o) + '\n')

In [17]:
%%cmd

python -m pyserini.index.lucene ^
    -input temp-data ^
    -collection JsonCollection ^
    -generator DefaultLuceneDocumentGenerator ^
    -index indexes/train_timeline_index ^
    -threads 1 ^
    -storePositions ^
    -storeDocvectors ^
    -storeRaw ^
    -language en

Microsoft Windows [Version 10.0.22631.3155]
(c) Microsoft Corporation. All rights reserved.

(clef) c:\Users\luisk\Projects-Win\clef-2024-authority\clef>
(clef) c:\Users\luisk\Projects-Win\clef-2024-authority\clef>python -m pyserini.index.lucene ^
More?     -input temp-data ^
More?     -collection JsonCollection ^
More?     -generator DefaultLuceneDocumentGenerator ^
More?     -index indexes/train_timeline_index ^
More?     -threads 1 ^
More?     -storePositions ^
More?     -storeDocvectors ^
More?     -storeRaw ^
More?     -language en


2024-02-25 17:50:39,517 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2024-02-25 17:50:39,520 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2024-02-25 17:50:39,520 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2024-02-25 17:50:39,520 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: temp-data
2024-02-25 17:50:39,520 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2024-02-25 17:50:39,520 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2024-02-25 17:50:39,520 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2024-02-25 17:50:39,520 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2024-02-25 17:50:39,521 INFO  [main] index.IndexCollection (IndexCollect

In [37]:
test_rumor = train_jsons[0]

In [41]:
search_input = test_rumor['rumor']
print(search_input)

“#Urgent / Ramallah Ministry of Health spokesman Kamal Al-Shakhra: We received 2,000 doses of the American “Moderna” #Corona vaccine, and this batch will be designated for President Abbas, the Fatah Central Committee, and VIPs.”


In [39]:
from pyserini.search.lucene import LuceneSearcher

# if you get the error "NameError: name '_C' is not defined" --> restart the Jupyter Kernel

searcher = LuceneSearcher("indexes/train_timeline_index")
hits = searcher.search(search_input)
hit_ids = []
for i in range(0, 5):
    doc = searcher.doc(hits[i].docid)
    hit_ids += [str(hits[i].docid)]
    json_doc = json.loads(doc.raw())
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f} {json_doc["contents"]}')

 1 1356683511047028748 9.24720 Yesterday, the Ministry of Health announced that it had received 2,000 doses of Moderna vaccine, stressing that the priority in receiving anti-Coronavirus vaccinations is for personnel working in the health sector, as they are the group who have the most contact with patients and are most at risk of infection with the Corona virus. #modernavaccine #palestine #COVID19
 2 1356178479251861504 7.09700 Vaccination will begin in the middle of this month, and we will provide the Gaza Strip with a share of all the vaccines we receive. We expect the first batch of vaccines that the government has contracted to purchase to arrive at the end of this month of February, or the beginning of the following month. The Ministry of Health was directed to prepare a health protocol for the electoral process.
 3 1356178477138010115 6.70100 "During today’s Cabinet session: - The Supreme Emergency Committee recommends to His Excellency the President to extend the state of emerge

In [40]:
for evidence in test_rumor['evidence']:
    print(f'{"WAS FOUND" if evidence[1] in hit_ids else "NOT FOUND"}\t{evidence[1]} {evidence[2]}')

NOT FOUND	1357211717479116800 Pictures from the launch of the vaccination campaign against the #Coronavirus, starting with the medical and health teams in #Bethlehem. #vaccine #Palestine #COVID19 https://t.co/vnhGmoZtgC
NOT FOUND	1356683687870488576 The Ministry added that the second group that will receive the vaccine is the elderly over the age of 60 and those with chronic diseases, because if they are infected with the virus, they will be more susceptible to serious symptoms. #vaccine #modernavaccine #palestine #COVID19
WAS FOUND	1356683511047028748 Yesterday, the Ministry of Health announced that it had received 2,000 doses of Moderna vaccine, stressing that the priority in receiving anti-Coronavirus vaccinations is for personnel working in the health sector, as they are the group who have the most contact with patients and are most at risk of infection with the Corona virus. #modernavaccine #palestine #COVID19
NOT FOUND	1356683390733471746 The Minister of Health, Dr. #May_Alkaila,